# Tarea 5: Redes Recurrentes <br/> CC6204 Deep Learning, Universidad de Chile <br/> Hoja de Respuestas

## Nombre: 
Fecha de entrega: 30 de diciembre de 2020

In [ ]:
import os

import torch
import torchvision

from collections import Counter
from torchvision import transforms
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from tqdm import tqdm

# Aqui descargamos algunas funciones utiles para resolver la tarea
if not os.path.exists('utils.py'):
    !wget https://raw.githubusercontent.com/dccuchile/CC6204/master/2020/tareas/tarea5/utils.py -q --show-progress

In [ ]:
from utils import extract_text_from_set, extract_text_from_set, tokenize_text 
from utils import encode_sentences, pad_sequence_with_lengths, pad_sequence_with_images
from utils import TextDataset, CaptioningDataset

# Parte 1: Generación de texto

### Datos

In [ ]:
##############################################################################
# Todo este código sirve para descargar, preprocesar y dejar los datos
# listos para usar después. Después de ejecutar las dos celdas siguientes
# tendrás los datos en train_flickr_tripletset y similar para val y test
##############################################################################

folder_path = './data/flickr8k'
if not os.path.exists(f'{folder_path}/images'):
    print('*** Descargando y extrayendo Flickr8k, siéntese y relájese 4 mins...')
    print('****** Descargando las imágenes...')
    !wget https://s06.imfd.cl/04/CC6204/tareas/tarea4/Flickr8k_Dataset.zip -P $folder_path/images -q --show-progress 
    print('********* Extrayendo las imágenes...\n  Si te sale mensaje de colab, dale Ignorar\n')
    !unzip -q $folder_path/images/Flickr8k_Dataset.zip -d $folder_path/images
    print('*** Descargando anotaciones de las imágenes...')
    !wget http://hockenmaier.cs.illinois.edu/8k-pictures.html -P $folder_path/annotations -q --show-progress

print('Inicializando pytorch Flickr8k dataset')
full_flickr_set = torchvision.datasets.Flickr8k(root=f'{folder_path}/images/Flicker8k_Dataset', ann_file = f'{folder_path}/annotations/8k-pictures.html')

print('Creando train, val y test splits...')
train_flickr_set, val_flickr_set, test_flickr_set = [], [], []
for i, item in enumerate(full_flickr_set):
  if i<6000:
    train_flickr_set.append(item)
  elif i<7000:
    val_flickr_set.append(item)
  else:
    test_flickr_set.append(item)

print('Listo!')

#### Extrae los textos

In [ ]:
train_text = extract_text_from_set(train_flickr_set)
val_text = extract_text_from_set(val_flickr_set)
test_text = extract_text_from_set(test_flickr_set)

#### Genera los tokens

In [ ]:
tokenizer = get_tokenizer('spacy')
counter = Counter()  # para llevar la cuenta de los tokens y su ocurrencia

train_tokens, counter = tokenize_text(train_text, tokenizer, counter)
test_tokens, counter = tokenize_text(test_text, tokenizer, counter)
val_tokens, counter = tokenize_text(val_text, tokenizer, counter)

#### Define el vocabulario y agrega `<pad>` y `<sos>`

In [ ]:
vocab = list(counter.keys())
vocab.append('<pad>')
vocab.append('<sos>')
word2idx = {word: i for i, word in enumerate(vocab)}
pad_idx = word2idx['<pad>']

#### Convierte oraciones a ids y genera los dataset de entrenamiento

In [ ]:
train_sentences = encode_sentences(train_tokens, vocab, word2idx)
test_sentences = encode_sentences(test_tokens, vocab, word2idx)
val_sentences = encode_sentences(val_tokens, vocab, word2idx)

train_dataset = TextDataset(train_sentences)
test_dataset = TextDataset(test_sentences)
val_dataset = TextDataset(val_sentences)

Con todo lo anterior, además de tener los dataset para entrenamiento, podemos también obtener identificadores correspondientes a textos que nosotros decidamos, haciendo algo como lo siguiente:

In [ ]:
s1 = 'A woman holding a cup of tea.'
s2 = 'A man with a dog.'

S = [s1,s2]
tokens, _ = tokenize_text(S, tokenizer)
D = encode_sentences(tokens, vocab, word2idx)

print('tokens:', tokens)
print('ids:', D)

#### Creamos los data loaders (puedes cambiar el tamaño del batch si lo deseas)



In [ ]:
batch_size = 64

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, 
    collate_fn=lambda data_list: pad_sequence_with_lengths(data_list, pad_idx))
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, 
    collate_fn=lambda data_list: pad_sequence_with_lengths(data_list, pad_idx))
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, 
    collate_fn=lambda data_list: pad_sequence_with_lengths(data_list, pad_idx))

**IMPORTANTE**: Nuestros datasets y dataloaders consideran también los largos de las secuencias. El siguiente código obtiene el primer elemento del dataset y el primer elemento del dataloader de prueba. Nota que lo que entregan en ambos casos es un par: la primera componente del par tiene los datos (los índices) mientras que la segunda componente tiene información de los largos de las secuencias.

In [ ]:
d, length = test_dataset[0]
print('len(d):', len(d))
print('length:', length)

In [ ]:
# Obtiene un paquete desde el dataloader
for data in test_dataloader:
  D, Lengths = data
  break

print(D.size())
print(Lengths.size())

# La primera dimensión de D corresponde al largo
# máximo de las secuencias en el batch
assert D.size()[0] == torch.max(Lengths)

# La segunda dimensión de D corresponde al tamaño del
# batch, al igual que la dimensión de Lengths
assert D.size()[1] == batch_size 
assert Lengths.size()[0] == batch_size

## 1a) Red recurrente

In [ ]:
# Acá empieza tu código

class RedRecurrente(torch.nn.Module):
    def __init__(self, ...): # Piensa en todo lo que necesitas para incializar.
        # Crea las capas considerando al menos los puntos de arriba.
        pass    

    def forward(self, x, h_0=None):
        # Acá debes programar la pasada hacia adelante.
        # El vector h_0 deberías simplemente pasarlo directo
        # a tu red recurrente (RNN, o GRU, o LSTM) y será necesario
        # para trabajar en la sección (1c) y en la parte 2. 
        # También puedes usar dropout, batch normalization o lo que necesites.
        logits = None
        return logits

## 1b) Entrenamiento

In [ ]:
# Acá tu código para el loop de entrenamiento
# y los gráficos de la pérdida

## 1c) Generación de texto

In [ ]:
# Acá tu código para generar texto usando el modelo

def generate_sentence(model, init_sentence, ...):
  # Usa acá lo que necesites para crear una secuencia de
  # salida. Muy posiblemente tendrás que usar un tokenizador
  # y el diccionario para pasar de índices a tokens (palabras).
  sentence = None
  return sentence

## 1d) Opcional: Beam Search

In [ ]:
# Acá tu código para generar texto usando beam search

def beam_search_generation(model, init_sentence, K, ...):
  # El K representa al ancho del beam para la búsqueda.
  return sentence

# Parte 2 (Opcional): Subtitulado de imágenes

#### Generamos transformación para el dataset

Algo importante es que estamos usando la normalización estándar para los modelos pre-entrenados que provee pytoch. Si vas a usar algún otro modelo (o incluso uno generado por ti), podrías necesitar otra normalización. También nota que estamos usando el tamaño estándar de `224x224` para las imágenes que reciben los modelos pre-entrenados de pytorch. Si no quieres usar esos modelos o si quieres hacer el entrenamiento más rápido, puedes cambiarle la resolución a las imágenes.

In [ ]:
transform = transforms.Compose(
            [
              transforms.ToTensor(), 
              transforms.Resize((224, 224)),
              transforms.Normalize(
                  mean=[0.485, 0.456, 0.406], 
                  std=[0.229, 0.224, 0.225])
            ])

#### Creamos los data loaders (puedes cambiar el tamaño del batch si lo deseas)

In [ ]:
batch_size = 16

train_dataloader = DataLoader(
    CaptioningDataset(
        train_flickr_set, transform, tokenizer, word2idx, "<sos>", "."),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda x: pad_sequence_with_images(x, pad_idx)
    )

test_dataloader = DataLoader(
    CaptioningDataset(
        test_flickr_set, transform, tokenizer, word2idx, "<sos>", "."),
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda x: pad_sequence_with_images(x, pad_idx)
    )

val_dataloader = DataLoader(
    CaptioningDataset(
        val_flickr_set, transform, tokenizer, word2idx, "<sos>", "."),
    batch_size=batch_size,
    shuffle=False,
    collate_fn=lambda x: pad_sequence_with_images(x, pad_idx)
    )

**IMPORTANTE**: Nuestros dataloaders ahora contienen las secuencias de identificadores de los tokens del texto, los largos de las secuencias y las imágenes correspondientes. El siguiente código obtiene el primer elemento del dataloader de prueba. Nota que lo que entregan en ambos casos es una tripleta: la primera componente tiene los datos desde los textos (los índices), la segunda componente tiene información de los largos de las secuencias, y la tercera componente la información de las imágenes.

In [ ]:
# Obtiene un paquete desde el dataloader
for data in test_dataloader:
  Text, Lengths, Img = data
  break

print(Text.size())
print(Lengths.size())
print(Img.size())

# La primera dimensión de Text corresponde al largo
# máximo de las secuencias en el batch
assert Text.size()[0] == torch.max(Lengths)

# La segunda dimensión de D corresponde al tamaño del
# batch, al igual que la dimensión de Lengths y la primera
# dimensión de Img
assert Text.size()[1] == batch_size 
assert Lengths.size()[0] == batch_size
assert Img.size()[0] == batch_size

### Usando modelos pre-entrenados

El siguiente código carga VGG16 (pre-entrenado), pasa el modelo a la GPU. 

In [ ]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
vgg16 = vgg16.to('cuda')

Con un codigo como el siguiente podemos calcular las características para las imágenes `Img` del batch que obtuvimos más arriba. Nota el uso de `.eval()` y `with torch.no_grad()`.

In [ ]:
Img = Img.to('cuda')

vgg16.eval()
with torch.no_grad():
  F = vgg16.features(Img)

print(F.size())

Finalmente y por si lo necesitas, puedes acceder a las imágenes originales del dataloader haciendo algo como esto:

In [ ]:
val_dataloader.dataset.original_image(0)

## 2a) Red convolucional + recurrente

In [ ]:
class CaptioningModel(torch.nn.Module):
    def __init__(self, ...): 
        # Crea las capas considerando una parte que procese debe procesar
        # la imagen de entrada y otra que debe producir el texto (índices)
        # de salida.
        pass
        
    def forward(self, ...):
        # Acá debes programar la pasada hacia adelante.
        # Debes decidir qué le pasarás a la red y cómo haras la 
        # computación hacia adelante. Considera que no solo
        # debes entrenar los parámetros sino que además debes
        # después ser capaz de generar una secuencia de salida
        # desde una imagen de entrada.
        return ...   

## 2b) Entrenamiento

In [ ]:
# Acá tu código para el loop de entrenamiento
# y los gráficos de la pérdida

## 2c) Generando texto desde imágenes de prueba


In [ ]:
# Acá tu código para generar texto usando desde imágenes
# y un par de ejemplos con las imágenes del conjunto de prueba